In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lupus_(Systemic_Lupus_Erythematosus)"
cohort = "GSE165004"

# Input paths
in_trait_dir = "../../input/GEO/Lupus_(Systemic_Lupus_Erythematosus)"
in_cohort_dir = "../../input/GEO/Lupus_(Systemic_Lupus_Erythematosus)/GSE165004"

# Output paths
out_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/GSE165004.csv"
out_gene_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/gene_data/GSE165004.csv"
out_clinical_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/clinical_data/GSE165004.csv"
json_path = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Endometrial Tissue RNA expression in recurrent pregnancy losses and unexplained infertility vs. conrol"
!Series_summary	"Recent studies are directed to decode the genetic signature of endometrial receptivity for better outcomes in assisted reproductive technologies. In this study, we aimed to understand the transcriptomic profile of midsecretory phase endometria of patients with recurrent pregnancy losses (RPL) and unexplained infertility (UI) by comparing with the endometria of healthy fertile women (Controls)."
!Series_summary	"In this prospective cohort study, we took endometrial samples from 24 patients with RPL, 24 patients with UI and 24 Controls at day 19-21 of the menstrual cycle. By performing genomic analysis, we assessed for differentially expressed genes (DEGs) and pathway analysis."
!Series_overall_design	"All patients involved in this prospective cohort study were recruited from Istanbul University School of Medicine between August 2

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Load the data from the previous step
clinical_data = pd.DataFrame({'sample': range(1, 73)})  # Creating a placeholder DataFrame
for i, values in {0: ['subject status/group: Control', 'subject status/group: patient with RPL', 'subject status/group: patient with UIF'], 1: ['tissue: Endometrial tissue']}.items():
    clinical_data[i] = values[0]  # Just fill with the first value, to be modified next

# 1. Gene Expression Data Availability
# Based on the background information, this appears to be a gene expression study analyzing
# differentially expressed genes in endometrial tissue, so we should have gene expression data
is_gene_available = True

# 2.1 Data Availability
# This dataset is about recurrent pregnancy losses and unexplained infertility,
# not Lupus (Systemic Lupus Erythematosus). There is no Lupus trait data here.
trait_row = None

# Age is not mentioned in the sample characteristics
age_row = None

# Gender is not explicitly mentioned, but from the background this is a study on female reproductive health
# (endometrial tissue), so all subjects are female, making it a constant feature
gender_row = None

# 2.2 Data Type Conversion functions
# These functions are included for completeness but won't be used since the required data is not available
def convert_trait(value):
    """
    Convert trait value to binary (0 for control, 1 for disease)
    """
    if ":" in value:
        value = value.split(":")[1].strip()
    
    # The dataset doesn't contain information about Lupus
    return None

def convert_age(value):
    """
    Convert age value to continuous number
    """
    if ":" in value:
        value = value.split(":")[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """
    Convert gender value to binary (0 for female, 1 for male)
    """
    if ":" in value:
        value = value.split(":")[1].strip()
    
    if value.lower() in ["female", "f"]:
        return 0
    elif value.lower() in ["male", "m"]:
        return 1
    else:
        return None

# 3. Save Metadata
# Trait data is NOT available for Lupus in this dataset
is_trait_available = trait_row is not None

# Save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this substep since trait_row is None (no Lupus trait data available)
# The dataset is studying pregnancy loss and infertility, not Lupus


False

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 62976
First 20 gene/probe identifiers:
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers shown, these appear to be numeric indices (1, 2, 3, etc.) rather than
# human gene symbols or standard identifiers like Entrez IDs or Ensembl IDs.
# 
# These numeric identifiers need to be mapped to actual gene symbols for meaningful analysis.
# In GEO datasets, such numeric identifiers often correspond to probe IDs that require mapping
# to gene symbols using platform annotation files.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'GB_ACC': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'LOCUSLINK_ID': [nan, nan, nan, 50865.0, 23704.0], 'GENE_SYMBOL': [nan, nan, nan, 'HEBP1', 'KCNE4'], 'GENE_NAME': [nan, nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.642618', 'Hs.348522'], 'ENSEMBL_ID': [nan, nan, nan, 'ENST00000014930', 'ENST00000281830'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_015987|ens|ENST00000014930|gb|AF117615|gb|BC016277', 'ref|NM_080671|ens|ENST00000281830|tc|THC2655788'], 'CHROMOSOMAL_LOCATION': 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the right columns for mapping between gene expression data and gene annotations
# The gene expression data has numeric indices that match the 'ID' column in gene_annotation
# The 'GENE_SYMBOL' column in gene_annotation contains the human gene symbols we want to map to

# 2. Create the gene mapping dataframe using the appropriate columns
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'GENE_SYMBOL')
print("Gene mapping dataframe created with shape:", gene_mapping.shape)
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level expression to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print("Gene expression data after mapping:")
print("Number of genes:", len(gene_data))
print("Number of samples:", len(gene_data.columns))
print("First few genes and samples:")
print(preview_df(gene_data))


Gene mapping dataframe created with shape: (54295, 2)
Gene mapping preview:
{'ID': ['4', '5', '6', '7', '8'], 'Gene': ['HEBP1', 'KCNE4', 'BPIFA3', 'LOC100129869', 'IRG1']}
Gene expression data after mapping:
Number of genes: 20353
Number of samples: 72
First few genes and samples:
{'GSM5024320': [16.175948293, 7.516155648, 46.507521084, 15.217321422, 149.14750401999999], 'GSM5024321': [16.011941074, 8.217093594, 48.623684219, 17.167041386, 154.14509097], 'GSM5024322': [16.572305297, 8.622670281, 49.566614717, 16.896298113, 160.74762947], 'GSM5024323': [15.295619113, 7.726044368, 50.172972701, 17.398087139, 147.61739465], 'GSM5024324': [15.083688478, 7.957276484, 48.823144464, 16.810108987, 147.75492186], 'GSM5024325': [16.328166471, 8.197432653, 50.503169901999996, 14.990370204000001, 146.82150985], 'GSM5024326': [16.823341796, 8.457233429, 46.53437868, 17.392785906, 158.77921269], 'GSM5024327': [16.669563058, 8.552464361, 48.166319294000004, 15.09498574, 153.30795133], 'GSM5024328': [

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print(f"Top 10 gene indices before normalization: {gene_data.index[:10].tolist()}")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Top 10 gene indices after normalization: {normalized_gene_data.index[:10].tolist()}")
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")

# Create directory for gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# Based on our analysis in Step 2, we determined that this dataset does not contain 
# lupus trait data. The dataset is about recurrent pregnancy losses and unexplained infertility,
# not Lupus (Systemic Lupus Erythematosus).

# Create a dummy clinical dataframe to properly document the absence of trait data
sample_ids = normalized_gene_data.columns.tolist()
clinical_df = pd.DataFrame(index=sample_ids)
# We'll create an empty clinical dataframe to accurately represent that we don't have lupus data
print(f"Clinical data not available for {trait} in this dataset")

# 3. Prepare linked data - in this case, it will just be the gene expression data
# with missing clinical information
linked_data = normalized_gene_data.T
print(f"Shape of linked data: {linked_data.shape}")

# 4. We cannot properly handle missing values for trait because the trait data 
# doesn't exist in this dataset

# 5. Since the trait data is completely missing, we shouldn't proceed with bias analysis
is_trait_biased = True  # Mark as biased because we have no trait data for lupus
unbiased_linked_data = linked_data  # Keep the gene data for reference

# 6. Conduct quality check and save the cohort information - report that trait data is not available
is_trait_available = False  # No trait data for lupus
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from endometrial tissue study of recurrent pregnancy losses " +
         "and unexplained infertility, not lupus. No lupus trait data available."
)

# 7. Since the linked data is not usable for lupus analysis, we won't save it as a final dataset
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed: No lupus trait data available. Final linked data not saved.")

Top 10 gene indices before normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


Top 10 gene indices after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06', 'AAA1']
Shape of normalized gene data: (19847, 72)


Saved normalized gene data to ../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/gene_data/GSE165004.csv
Clinical data not available for Lupus_(Systemic_Lupus_Erythematosus) in this dataset
Shape of linked data: (72, 19847)
Dataset validation failed: No lupus trait data available. Final linked data not saved.
